# Imports

In [20]:
import torch
import pandas 
import numpy as np
from torchsummary import summary

In [2]:
training_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("USING DEVICE:", training_device)

USING DEVICE: cuda


# Setting up the Dataset

In [3]:
import pandas as pd
df_noise=pd.read_csv("Data/cf_train.csv")
df_no_noise=pd.read_csv("Data/cf_train_no_noise.csv")

In [4]:
class CustomDataset:
    def __init__(self,dataframe,batch_size,device =training_device,shuffle=False):
        self.df=dataframe
        self.batch_size=batch_size
        self.columns_to_drop=['row_num','day','era','target_10_val','target_5_val','sigma','day_no']
        self.X = self.df.drop(self.columns_to_drop, axis=1)
        self.y=self.df['target_10_val']
        self.device=device
        self.shuffle=shuffle

    def generate_batches_with_labels(self,idx):
        data=self.X.iloc[:max(0,idx-10)]
        labels=self.y.iloc[:max(0,idx-10)]
        dataset =  torch.utils.data.TensorDataset(torch.tensor(data.values),torch.tensor(labels.values))
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=self.shuffle)
        data_unseen = self.X.iloc[max(0,idx-9):idx+1]
        labels_unseen=self.y.iloc[max(0,idx-9):idx+1]
        data_unseen,labels_unseen = torch.tensor(data_unseen.values).to(self.device),torch.tensor(labels_unseen.values).to(self.device)
        return dataloader, (data_unseen,labels_unseen)

# Test Time Adaptation Model

In [48]:
class TestTimeModelMLP(torch.nn.Module):
    def __init__(
        self,
        input_size,
        classification_output_size,
        shared_layers,
        classification_layers,
        auxiliary_layers,
    ):
        super(TestTimeModelMLP, self).__init__()
        self.input_size = input_size
        shared = []
        for i in range(len(shared_layers)):
            if i == 0:
                shared.append(torch.nn.Linear(input_size, shared_layers[i]))
                shared.append(torch.nn.ReLU())
            elif i < len(shared_layers) - 1:
                shared.append(torch.nn.Linear(shared_layers[i - 1], shared_layers[i]))
                shared.append(torch.nn.ReLU())
            else:
                shared.append(torch.nn.Linear(shared_layers[i - 1], shared_layers[i]))
                shared.append(torch.nn.ReLU())
                shared.append(torch.nn.BatchNorm1d(shared_layers[i]))

        self.shared_layers = torch.nn.Sequential(*shared)

        classification = []
        for i in range(len(classification_layers)):
            if i == 0 and len(classification_layers) == 1:
                classification.append(torch.nn.Linear(shared_layers[-1], classification_output_size))
            elif i == 0:
                classification.append(
                    torch.nn.Linear(shared_layers[-1], classification_layers[i])
                )
                classification.append(torch.nn.ReLU())
            elif i < len(classification_layers) - 1:
                classification.append(
                    torch.nn.Linear(
                        classification_layers[i - 1], classification_layers[i]
                    )
                )
                classification.append(torch.nn.ReLU())
                classification.append(torch.nn.BatchNorm1d(classification_layers[i]))
            else:
                classification.append(
                    torch.nn.Linear(classification_layers[i - 1], classification_output_size)
                )

        self.classification_layers = torch.nn.Sequential(*classification)

        auxiliary = []
        for i in range(len(auxiliary_layers)):
            if i == 0 and len(auxiliary_layers) == 1:
                auxiliary.append(torch.nn.Linear(shared_layers[-1], self.input_size))
            elif i == 0:
                auxiliary.append(
                    torch.nn.Linear(shared_layers[-1], auxiliary_layers[i])
                )
                auxiliary.append(torch.nn.ReLU())
            elif i < len(auxiliary_layers) - 1:
                auxiliary.append(
                    torch.nn.Linear(auxiliary_layers[i - 1], auxiliary_layers[i])
                )
                auxiliary.append(torch.nn.ReLU())
                auxiliary.append(torch.nn.BatchNorm1d(auxiliary_layers[i]))
            else:
                auxiliary.append(
                    torch.nn.Linear(auxiliary_layers[i - 1], self.input_size)
                )

        self.auxiliary_layers = torch.nn.Sequential(*auxiliary)

    def forward(self, x):
        shared_out = self.shared_layers(x)
        classification_out = self.classification_layers(shared_out)
        auxiliary_out = self.auxiliary_layers(shared_out)
        return classification_out, auxiliary_out

# Model Parameters & Training

In [49]:
tta_model = TestTimeModelMLP(input_size=24, classification_output_size=5, shared_layers=[64, 64], classification_layers=[64,64], auxiliary_layers=[64,64])
tta_model = tta_model.to(training_device)

In [50]:
tta_model

TestTimeModelMLP(
  (shared_layers): Sequential(
    (0): Linear(in_features=24, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (classification_layers): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=5, bias=True)
  )
  (auxiliary_layers): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=24, bias=True)
  )
)